# EDSR

In [1]:
'''
Modified By Sohaib Anwaar

This code is modified for tensorflow 2.

Hardware and software Compatibility:

    3070 RTX nvidia
    Intel 10400F processor (CPU)
    Ubuntu 18.04
    tensorflow 2.0
'''





import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
import os
import matplotlib.pyplot as plt

from data import DIV2K
from model.edsr import edsr
from train import EdsrTrainer

%matplotlib inline

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6



In [2]:
# Number of residual blocks
depth = 16

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [3]:
# Location of model weights (needed for demo)
weights_dir = f'weights/edsr-{depth}-x{scale}'
weights_file = os.path.join(weights_dir, 'weights.h5')

os.makedirs(weights_dir, exist_ok=True)

dataset_path = ""

images_dir='/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images'
caches_dir='/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/caches'

## Datasets

You don't need to download the DIV2K dataset as the required parts are automatically downloaded by the `DIV2K` class. By default, DIV2K images are stored in folder `.div2k` in the project's root directory.

In [4]:
div2k_train = DIV2K(images_dir= images_dir, caches_dir=caches_dir, scale=scale, subset='train', downgrade=downgrade)
div2k_valid = DIV2K(images_dir= images_dir, caches_dir=caches_dir,scale=scale, subset='valid', downgrade=downgrade)

In [5]:
train_ds = div2k_train.dataset(batch_size=16, random_transform=True)
valid_ds = div2k_valid.dataset(batch_size=1, random_transform=False, repeat_count=1)

['/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/15_HGE_Segx4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/36x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/5x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/8x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/Y28x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/26_HGE_Segx4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/19x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_train_LR_bicubic/X4/no_9x4.png', '/media/sohaib/additional_/DataScience/super_r

['/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/25x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/4x4x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/28_HGE_Segx4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/21_HGE_Segx4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/30_HGE_Segx4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/6_HGE_Segx4x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/10_HGE_Segx4x4.jpg', '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/Y188x4.jpg', '/media/sohaib/ad

## Training

### Pre-trained models

If you want to skip training and directly run the demo below, download [weights-edsr-16-x4.tar.gz](https://martin-krasser.de/sisr/weights-edsr-16-x4.tar.gz) and extract the archive in the project's root directory. This will create a `weights/edsr-16-x4` directory containing the weights of the pre-trained model.

In [6]:
trainer = EdsrTrainer(model=edsr(scale=2, num_res_blocks=depth), 
                      checkpoint_dir=f'.ckpt/edsr-{depth}-xy{scale}')

In [7]:
# Train EDSR model for 300,000 steps and evaluate model
# every 1000 steps on the first 10 images of the DIV2K
# validation set. Save a checkpoint only if evaluation
# PSNR has improved.
trainer.train(train_ds,
              valid_ds.take(10),
              steps=300000, 
              evaluate_every=1000, 
              save_best_only=True)

ValueError: in user code:

    /media/sohaib/additional_/DataScience/super_resolution/super-resolution/train.py:81 train_step  *
        loss_value = self.loss(hr, sr)
    /home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:152 __call__  **
        losses = call_fn(y_true, y_pred)
    /home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:256 call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    /home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:1231 mean_absolute_error
        return K.mean(math_ops.abs(y_pred - y_true), axis=-1)
    /home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1180 binary_op_wrapper
        raise

In [ ]:
# Restore from checkpoint with highest PSNR
trainer.restore()

In [ ]:
# Evaluate model on full validation set
psnrv = trainer.evaluate(valid_ds)
print(f'PSNR = {psnrv.numpy():3f}')

In [ ]:
# Save weights to separate location (needed for demo)
trainer.model.save_weights(weights_file)

## Demo

In [ ]:
model = edsr(scale=scale, num_res_blocks=depth)
model.load_weights(weights_file)

In [ ]:
from model import resolve_single
from utils import load_image, plot_sample

def resolve_and_plot(lr_image_path):
    lr = load_image(lr_image_path)
    sr = resolve_single(model, lr)
    plot_sample(lr, sr)

In [ ]:
resolve_and_plot('demo/0869x4-crop.png')

In [ ]:
resolve_and_plot('demo/0829x4-crop.png')

In [ ]:
resolve_and_plot('demo/0851x4-crop.png')

In [ ]:
from tensorflow.python.data.experimental import AUTOTUNE

# -----------------------------------------------------------
#  Transformations
# -----------------------------------------------------------


def random_crop(lr_img, hr_img, hr_crop_size=96, scale=2):
    lr_crop_size = hr_crop_size // scale
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w * scale
    hr_h = lr_h * scale

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped


def random_flip(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=1)
    return tf.cond(rn < 0.5,
                   lambda: (lr_img, hr_img),
                   lambda: (tf.image.flip_left_right(lr_img),
                            tf.image.flip_left_right(hr_img)))


def random_rotate(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=4, dtype=tf.int32)
    return tf.image.rot90(lr_img, rn), tf.image.rot90(hr_img, rn)


# -----------------------------------------------------------
#  IO
# -----------------------------------------------------------


def download_archive(file, target_dir, extract=True):
    source_url = f'http://data.vision.ee.ethz.ch/cvl/DIV2K/{file}'
    target_dir = os.path.abspath(target_dir)
    tf.keras.utils.get_file(file, source_url, cache_subdir=target_dir, extract=extract)
    os.remove(os.path.join(target_dir, file))


def dataset(batch_size=16, repeat_count=None, random_transform=True):
    
    hr = div2k_train.hr_dataset()
    lr = div2k_train.lr_dataset()
    ds = tf.data.Dataset.zip((lr, hr))
#     if random_transform:
#         ds = ds.map(lambda lr, hr: random_crop(lr, hr, scale=4), num_parallel_calls=AUTOTUNE)
#         ds = ds.map(random_rotate, num_parallel_calls=AUTOTUNE)
#         ds = ds.map(random_flip, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.repeat(repeat_count)
#         ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

df = dataset(batch_size=16, repeat_count=None, random_transform=True)

In [ ]:
for lr, hr in df.take(1):
    print(lr)

In [ ]:
hr = div2k_train.hr_dataset()
lr = div2k_train.lr_dataset()


for i in hr.take(1):
    print(i.numpy().shape)
    

for i in lr.take(1):
    print(i.numpy().shape)

In [8]:
from PIL import Image

Image.open('/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/20_HGE_Segx4.jpg').size

FileNotFoundError: [Errno 2] No such file or directory: '/media/sohaib/additional_/DataScience/super_resolution/dataset/dataset3/images/DIV2K_valid_LR_bicubic/X4/20_HGE_Segx4.jpg'